## Bitcoin Allocation Strategies
Comparing different allocation strategies historically

. Lump-Sum

. Dollar Cost Averaging (DCA)

. Combination (optimized)

In [383]:
# Import Libraries
import pandas as pd
import numpy as np
import urllib
import requests
import riskfolio as rp
from datetime import datetime, timedelta
from copy import deepcopy

# Chart libraries + settings
%matplotlib inline
import matplotlib.pyplot as plt
# plt.style.use('seaborn-whitegrid')
pd.options.mode.chained_assignment = None  # default='warn' - disable some pandas warnings


In [384]:
# Load Bitcoin Prices into a dataframe
# Ticker is customizable
ticker = "BTC"
# Cryptocompare URL and fiels
base_url = 'https://min-api.cryptocompare.com/data/histoday'
ticker_field = 'fsym'
field_dict = {'tsym': 'USD','allData': 'true'}
# Convert the field dict into a url encoded string
url_args = "&" + urllib.parse.urlencode(field_dict)
ticker = ticker.upper()
globalURL = (base_url + "?" + ticker_field + "=" + ticker + url_args)


In [385]:
# Request the data
resp = requests.get(url=globalURL)
data = resp.json()
data["Response"]

'Success'

In [386]:
# Parse the JSON into a Pandas DataFrame
try:
    df = pd.DataFrame.from_dict(data['Data'])
    df = df.rename(columns={'time': 'date'})
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df.set_index('date', inplace=True)
    df_save = df[['close', 'open', 'high', 'low']]
except Exception as e:
    print(e)
    df_save = None

In [387]:
# Include percentage change 
df = df_save
df['change'] = df['close'].pct_change()

### Support Utilities to be used later

In [388]:
# Increment n number of months of certain date
def monthdelta(date, delta):
    m, y = (date.month+delta) % 12, date.year + ((date.month)+delta-1) // 12
    if not m: m = 12
    d = min(date.day, [31,
        29 if y%4==0 and not y%400==0 else 28,31,30,31,30,31,31,30,31,30,31][m-1])
    new_date = (date.replace(day=d,month=m, year=y))
    return new_date

def add_periods(date, periods, frequency):
    if frequency.upper() == 'D' or 'DAY' in frequency.upper():
        return (date + timedelta(days=periods))
    if frequency.upper() == 'W' or 'WEEK' in frequency.upper():
        return (date + timedelta(days=periods * 7))
    if frequency.upper() == 'M' or 'MONTH' in frequency.upper():
        return(monthdelta(date, periods))
    if frequency.upper() == 'Y' or 'YEAR' in frequency.upper():
        return(monthdelta(date, periods * 12))
    
def annualization_factor(df):
    """
    Receives a df and returns the number of periods to apply
    to annualize the returns. For BTC this should be close to
    365 as it trades daily. For stocks should be close to 252.
    Args:
        df (_type_): _description_
    """
    start_date = df.index[0]
    end_date = df.index[-1]
    number_of_days = (end_date - start_date).days
    fraction_of_year = number_of_days / 365
    data_points = len(df)
    annualization_factor = data_points / fraction_of_year
    return int(round(annualization_factor, 0))


### Main Allocation Class
See example on creating an allocation instance at the cell following the class definition

In [442]:
class AllocationManager:
    def __init__(self):
        self.frequency = 'D'  # 'D', 'W', 'M', 'Y'
        self.allocation_periods = 30  # Assume allocation happens during 30 periods
        self.upfront_percent = 0  # [0 - 1]: amount to be allocated upfront
        self.capital = 100000  # 10,000 dollars to allocate
        self.df = df  # Bitcoin Prices Dataframe
        self.start_date = self.df.index.min()  # Date where allocation starts (default = first date)
                                               # ex: pd.to_datetime('2023-01-01', format='%Y-%m-%d')
        self.end_date = self.df.index.max()  # End date for analysis (default = today), but this can be used to test specific timeframes (ex: ending last year)
                                             # ex: pd.to_datetime('2023-01-01', format='%Y-%m-%d')
        # Create empty allocation & position columns
        self.df['allocation'] = 0
        self.df['BTC_tx'] = 0
        self.risk_free_rate = 0.05
         
    def allocate_capital(self):
        # TRIM THE DF between start and end dates
        # Filter the dataframe to only include selected dates
        self.df = self.df[(self.df.index >= (self.start_date)) & (self.df.index <= self.end_date)]
        
        # Updates the dataframe to allocate the capital
        available_capital = self.capital 
        current_date = self.start_date
        periods_left = self.allocation_periods
        
        # Set upfront amount if any & per period amounts
        if self.upfront_percent > 0:
            upfront = self.upfront_percent * self.capital  # how much upfront in $
            per_period = (self.capital - upfront) / (self.allocation_periods - 1)
        else:
            per_period = self.capital / self.allocation_periods 
            upfront = per_period
        
        # Start looping until allocation is complete
        while periods_left > 0:
            # Allocate Capital
            if current_date == self.start_date:
                self.df.at[current_date, 'allocation'] = upfront
            else:
                self.df.at[current_date, 'allocation'] = per_period
            
            # Allocate BTC
            self.df.at[current_date, 'BTC_tx'] = (
                self.df.at[current_date, 'allocation'] / 
                self.df.at[current_date, 'close'] 
                )
            current_date = add_periods(current_date, 1, self.frequency)
            if current_date > self.end_date:
                raise Exception("Allocation dates overflow end date. Either shorten the allocation period or increase the data range.")
            periods_left -= 1

        # Sum all BTC Txs and calculate portfolio values
        self.df['BTC_position'] = self.df['BTC_tx'].cumsum()
        self.df['portfolio_position'] = (self.df['BTC_position'] * self.df['close'])
        self.df = self.df.fillna(0)
        # TO DO ---------------------
        self.df['cum_capital'] = self.df['allocation'].cumsum()
        self.df['normalized_port_position'] = (self.df['portfolio_position'] / 
                                               self.df['cum_capital']) * 100
        self.df['cum_return'] = self.df['normalized_port_position'] / 100

    def show_allocations(self):
        al_df = self.df.where(self.df.allocation > 0).dropna()
        return (al_df)
        
    def stats(self):
        df = self.df
        stats = {}
        stats = stats | deepcopy(self.__dict__)
        del stats['df'] 
        stats['capital allocated'] = df.allocation.sum()
        stats['BTC allocated'] = df.BTC_tx.sum()
        stats['max portfolio value'] = df.portfolio_position.max()
        stats['final portfolio value'] = df.portfolio_position[-1]
        # Calculate Return on Invested Capital
        stats['ROIC'] = (
            (stats['final portfolio value'] / 
             stats['capital allocated']) - 1
            )
        # Calculate Multiple of Invested Capital
        stats['MOIC'] = (
            (stats['final portfolio value'] / 
             stats['capital allocated'])
            )
        
        stats['number_points'] = df['change'].count()
        stats['volatility_daily'] = df['change'].std()
        stats['annualization_factor'] = annualization_factor(df)
        stats['volatility_annual'] = df['change'].std() * annualization_factor(df)**.5
        stats['return_annual'] = ((df['cum_return'][-1]) ** (annualization_factor(df) / df['change'].count())) -1
    
        stats['sharpe_ratio'] = (stats['return_annual'] -
                             self.risk_free_rate) / stats['volatility_annual']
        Y = df['normalized_port_position'].pct_change().dropna()
        stats['hist_VaR_95'] = rp.RiskFunctions.VaR_Hist(Y, alpha=0.05)
        stats['max_DD_abs'] = rp.RiskFunctions.MDD_Rel(Y)
        stats['avg_DD_abs'] = rp.RiskFunctions.ADD_Rel(Y)
        stats['DaR_DD_abs_95'] = rp.RiskFunctions.DaR_Rel(Y, alpha=0.05)
        # Bitcoin stats
        stats['BTC_initial'] = df['close'][0]
        stats['BTC_final'] = df['close'][-1]
        stats['BTC_MOIC'] = df['close'][-1] / df['close'][0]
        stats['BTC_period'] = (stats['BTC_final'] / stats['BTC_initial']) - 1
        
        # Compared to BTC - how much more or less $100 invested in the 
        # allocation strategy resulted compared to $100 in BTC
        stats['outperformance_DV100'] = (stats['MOIC'] * 100) - (stats['BTC_MOIC'] * 100)
        
        #  Cost Basis Statistics
        stats['weighted_avg_cost'] =  (stats['capital allocated'] / 
                                       stats['BTC allocated'])
        txs = self.show_allocations()
        stats['avg_buy_price'] = txs['close'].mean()
        stats['max_buy_price'] = txs['close'].max()
        stats['min_buy_price'] = txs['close'].min()
        stats['first_buy_price'] = txs['close'][0]
        stats['lump_sum_beats_avg'] = (stats['first_buy_price'] < stats['avg_buy_price'])
        return (stats)
    
def run_through_time(alloc):
    # Do the same allocation for every start date 
    # in the dataframe. This basically shortens the dataframe by 1 day
    # on every loop and stores the results.
    # And stores the results in a new df with stats.
    
    # Stores initial values
    stats = {}
    stats = stats | deepcopy(alloc.__dict__)
    del stats['df'] 
    
    tmp_alloc = deepcopy(alloc)  # Creates a copy of Alloc Instance
    error = None
    results = []
    
    # Will loop from start date until there aren't enough periods to allocate 
    while True:
        try:
            tmp_alloc.allocate_capital()
            results.append(tmp_alloc.stats())
            tmp_alloc.start_date = add_periods(tmp_alloc.start_date, 1, 'D')
        except Exception as e:
            break
    df = pd.DataFrame(results)
    
    stats['number_of_allocations'] = df['frequency'].count()
    
    return (df, stats)
        
        

In [436]:
# Create a sample Instance of the AllocationManager and test results
btc_alloc = AllocationManager()
btc_alloc.capital = 100
btc_alloc.allocation_periods = 10
btc_alloc.frequency = 'D'
btc_alloc.start_date = pd.to_datetime('2022-01-01', format='%Y-%m-%d')
btc_alloc.upfront_percent = 0
btc_alloc.allocate_capital()

In [429]:
btc_alloc.allocate_capital()

In [434]:
# Show statistics
btc_alloc.stats()

{'frequency': 'D',
 'allocation_periods': 10,
 'upfront_percent': 0,
 'capital': 100,
 'start_date': Timestamp('2022-01-01 00:00:00'),
 'end_date': Timestamp('2023-01-19 00:00:00'),
 'risk_free_rate': 0.05,
 'capital allocated': 100,
 'BTC allocated': 0.0022751424756657134,
 'max portfolio value': 107.94481730070248,
 'final portfolio value': 47.13130549169676,
 'ROIC': -0.5286869450830325,
 'MOIC': 0.47131305491696757,
 'number_points': 384,
 'volatility_daily': 0.03315284379200532,
 'annualization_factor': 366,
 'volatility_annual': 0.6342512474153603,
 'return_annual': -0.5117715442670766,
 'sharpe_ratio': -0.8857239880195018,
 'hist_VaR_95': 0.05737567486829054,
 'max_DD_abs': 0.6678239994267082,
 'avg_DD_abs': 0.40445511542256773,
 'DaR_DD_abs_95': 0.6491106600653807,
 'BTC_initial': 47737.35,
 'BTC_final': 20715.76,
 'BTC_MOIC': 0.4339528691894292,
 'BTC_period': -0.5660471308105708,
 'outperformance_DV100': 3.736018572753835,
 'weighted_avg_cost': 43953.29130793873,
 'avg_buy_pr

In [409]:
btc_alloc.df

,close,open,high,low,change,allocation,BTC_tx,BTC_position,portfolio_position,cum_capital,normalized_port_position,cum_return
date,,,,,,,,,,,,
2023-01-01,16613.87,16531.31,16619.82,16502.85,0.004994,10,0.000602,0.000602,10.000000,10,100.000000,1.000000
2023-01-02,16670.08,16613.87,16768.07,16544.15,0.003383,10,0.000600,0.001202,20.033833,20,100.169166,1.001692
2023-01-03,16670.16,16670.08,16766.70,16605.01,0.000005,10,0.000600,0.001802,30.033929,30,100.113098,1.001131
2023-01-04,16846.82,16670.16,16972.62,16651.02,0.010597,10,0.000594,0.002395,40.352210,40,100.880526,1.008805
2023-01-05,16825.87,16846.82,16869.84,16764.64,-0.001244,10,0.000594,0.002990,50.302030,50,100.604060,1.006041
2023-01-06,16946.16,16825.87,17013.77,16687.42,0.007149,10,0.000590,0.003580,60.661645,60,101.102741,1.011027
2023-01-07,16942.73,16946.16,16972.62,16905.39,-0.000202,10,0.000590,0.004170,70.649366,70,100.927666,1.009277
2023-01-08,17115.81,16942.73,17132.21,16913.28,0.010216,10,0.000584,0.004754,81.371091,80,101.713864,1.017139
2023-01-09,17179.03,17115.81,17387.59,17103.20,0.003694,10,0.000582,0.005336,91.671649,90,101.857387,1.018574


In [443]:
d, stats = run_through_time(btc_alloc)
stats

{'frequency': 'D',
 'allocation_periods': 10,
 'upfront_percent': 0,
 'capital': 100,
 'start_date': Timestamp('2022-01-01 00:00:00'),
 'end_date': Timestamp('2023-01-19 00:00:00'),
 'risk_free_rate': 0.05,
 'number_of_allocations': 374}

In [358]:
# Show only the allocation periods
al_df = btc_alloc.show_allocations()
display(al_df)
# Check it adds to allocation amount
print("Total allocation:")
print(round(al_df.allocation.sum(), 8))
if (round(al_df.allocation.sum(), 8) == round(btc_alloc.capital, 8)):
    print ("Checks [OK]")
else:
    print ("[ERROR] - something went wrong")

,close,open,high,low,change,allocation,BTC_tx,BTC_position,portfolio_position,cum_capital,normalized_port_position,cum_return
date,,,,,,,,,,,,
2023-01-01,16613.87,16531.31,16619.82,16502.85,0.004994,33.333333,0.002006,0.002006,33.333333,33.333333,100.000000,1.000000
2023-01-02,16670.08,16613.87,16768.07,16544.15,0.003383,33.333333,0.002000,0.004006,66.779444,66.666667,100.169166,1.001692
2023-01-03,16670.16,16670.08,16766.70,16605.01,0.000005,33.333333,0.002000,0.006006,100.113098,100.000000,100.113098,1.001131


Total allocation:
100.0
Checks [OK]


### Run Simulations
Create different allocation scenarios

In [319]:
# CONSTANTS
# Define some static variables that will remain constant

# --------------
# Start Date
# --------------
# Data from 2010 has less meaning than recent data. For this analysis we can use
# more recent data. Using only data since 2017.
start_date = pd.to_datetime('2017-01-01', format='%Y-%m-%d')

# Assuming $100,000 to be allocated
allocated_capital = 100000

# Risk Free Rate (used to calculate sharpe ratio)
risk_free_rate = 0.05


In [321]:
# Variables
# These will be the ranges / variables to be simulated

frequency = ['D', 'W', 'M']
allocation_periods = {
    'D': range(1, 30),
    'W': range(1, 8),
    'M': range(1, 4)
}
upfront_percent = [0, 0.1, 0.25, 0.4, 0.5, 0.75, 1]